In [1]:
import pandas as pd
import yfinance as yf

df_tickers = pd.read_csv('../data/sp500.csv')
tickers = df_tickers['Symbol'].to_list()
tickers = [ticker.replace('.', '-') for ticker in tickers]
period = "max"
interval = "1m"
df = yf.download(tickers, period=period, interval=interval)['Adj Close']
print(df.head())
# df['date'] = df.index

In [3]:
df.info()

In [4]:
df.columns

In [3]:
from utils.StockClassifier import StockClassifier
import yfinance as yf
import pandas as pd

# Example usage
# Define the stock tickers and S&P 500 ticker
tickers = ['AAPL', 'MSFT', 'GOOGL']  # Example tickers
sp500_ticker = '^GSPC'

# Download historical data for the stocks and the S&P 500 index
stock_data = yf.download(tickers, start='2020-01-01', end='2023-01-01')
sp500_data = yf.download(sp500_ticker, start='2020-01-01', end='2023-01-01')

# Close price and volume data
stock_df = stock_data['Close']
volume_df = stock_data['Volume']
sp500_df = sp500_data['Close']

print(stock_df.head())
print(volume_df.head())

# Initialize the StockClassifier
classifier = StockClassifier(stock_df, volume_df, sp500_df, tickers)

# Risk-based clustering
risk_clusters = classifier.risk_based_clustering()
print("Risk-based Clustering Results:")
print(risk_clusters)

# Performance-based clustering
performance_clusters = classifier.performance_clustering()
print("Performance-based Clustering Results:")
print(performance_clusters)

# Volume-based clustering
# volume_clusters = classifier.volume_based_clustering()
# print("Volume-based Clustering Results:")
# print(volume_clusters)

# Price-based clustering
# price_clusters = classifier.price_based_clustering()
# print("Price-based Clustering Results:")
# print(price_clusters)

# Example of fundamental data (e.g., P/E ratio, market cap)
fundamentals_data = pd.DataFrame({
    'Ticker': tickers,
    'P/E Ratio': [28.5, 35.2, 30.1],
    'Market Cap': [2.4e12, 1.8e12, 1.5e12]
}).set_index('Ticker')

# Fundamental-based clustering
fundamental_clusters = classifier.fundamental_based_clustering(fundamentals_data)
print("Fundamental-based Clustering Results:")
print(fundamental_clusters)

In [ ]:
print("Hello World")
import tensorflow as tf
import numpy as np
import gym

# Define the environment
env = gym.make('CartPole-v0')

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(24, input_shape=(4,), activation='relu'),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(2, activation='linear')
])


# Define the agent
class DQNAgent:
    def __init__(self, env, model):
        self.env = env
        self.model = model
        self.memory = []
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.batch_size = 32
        self.optimizer = tf.keras.optimizers.Adam(lr=0.001)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def choose_action(self, state):
        if np.random.rand() <= self.epsilon:
            return self.env.action_space.sample()
        else:
            return np.argmax(self.model.predict(state))

    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        batch = np.random.choice(self.memory, self.batch_size, replace=False)
        states = np.array([i[0] for i in batch])
        actions = np.array([i[1] for i in batch])
        rewards = np.array([i[2] for i in batch])
        next_states = np.array([i[3] for i in batch])
        dones = np.array([i[4] for i in batch])

        # Calculate Q values
        target = rewards + self.gamma * np.amax(self.model.predict(next_states), axis=1)
        target[dones] = rewards[dones]

        # Update the model
        self.model.fit(states, target, batch_size=self.batch_size, verbose=0)

        # Decay epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


# Initialize the agent
agent = DQNAgent(env, model)

# Train the agent
for episode in range(1000):
    state = env.reset()
    state = np.reshape(state, [1, 4])
    done = False
    score = 0
    while not done:
        # Choose action
        action = agent.choose_action(state)

        # Take action and observe next state and reward
        next_state, reward, done, info = env.step(action)
        next_state = np.reshape(next_state, [1, 4])

        # Remember the experience
        agent.remember(state, action, reward, next_state, done)

        # Update state
        state = next_state

        # Update score
        score += reward

    # Replay
    agent.replay()

    # Print score
    print("Episode: {}, Score: {}".format(episode, score))

# Test the agent
state = env.reset()
state = np.reshape(state, [1, 4])
done = False
score = 0
while not done:
    # Choose action
    action = agent.choose_action(state)

    # Take action and observe next state and reward
    next_state, reward, done, info = env.step(action)
    next_state = np.reshape(next_state, [1, 4])

    # Update state
    state = next_state

    # Update score
    score += reward

# Print final score
print("Final Score: {}".format(score))

# Close the environment
env.close()

In [6]:
import yfinance as yf

ticker = yf.Ticker('AAPL')
ticker.info

In [12]:

from datetime import datetime
from utils.StockDataManager import StockDataManager
import pandas as pd
import yfinance as yf
symbols = ["AAPL", "MSFT"]
period = '3mo'

stockDataManager = StockDataManager()

# Fetch historical data for the portfolio
portfolio_data_list = [stockDataManager.history(ticker=symbol, period=period, interval='1d')[['adj_close']] for symbol
                       in symbols]
portfolio_data = pd.concat(portfolio_data_list, axis=1, keys=[symbol for symbol in symbols])
portfolio_data.columns = portfolio_data.columns.droplevel(1)
portfolio_data = portfolio_data.ffill()
portfolio_data = portfolio_data.bfill()
mean_portfolio_data = portfolio_data.mean(axis=1)

# Convert dates to strings
mean_portfolio_data.index = mean_portfolio_data.index.astype(str)

# Fetch historical data for S&P 500
sp500_data = yf.download('^GSPC', start=datetime.strptime(portfolio_data.index.min(), '%Y-%m-%d').date(), end=datetime.strptime(portfolio_data.index.max(), '%Y-%m-%d').date(), interval='1d')['Adj Close']

# Convert dates to strings
sp500_data.index = sp500_data.index.astype(str)

In [14]:
datetime.strptime(portfolio_data.index.min(), '%Y-%m-%d').date()